In [1]:
import pandas as pd
import numpy as np

In [2]:
total_costs = pd.read_csv('Detergent_TotalCost.csv')
unit_costs = pd.read_csv('Detergent_UnitCost.csv')
price_indices = pd.read_csv('PriceIndices.csv')

In [4]:
unit_costs = unit_costs[['COICOP','GEO','Value']]
unit_costs_reformat = pd.DataFrame()
unit_costs_reformat['GEO'] = unit_costs['GEO'].unique()
for typeof in (unit_costs['COICOP'].unique()):
    specific_type = unit_costs[unit_costs['COICOP'] == typeof]
    specific_type = specific_type.reset_index(drop=True)
    unit_costs_reformat[typeof] = specific_type['Value']

In [5]:
count = 0
for i in range (0,len(unit_costs_reformat)):
    for j in range(1,len(unit_costs_reformat.columns)):
        if unit_costs_reformat.ix[i,j] == ':':
            count  = count + 1
    if (count==len(unit_costs_reformat.columns)-1):
        unit_costs_reformat = unit_costs_reformat.drop(i)
    count = 0
unit_costs_reformat = unit_costs_reformat.reset_index(drop=True)
total_costs = total_costs[total_costs['GEO'] != 'Euro area (EA11-1999, EA12-2001, EA13-2007, EA15-2008, EA16-2009, EA17-2011, EA18-2014, EA19-2015)']

C:\Users\patel\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.
C:\Users\patel\Anaconda3\lib\site-packages\pandas\core\indexing.py:961: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  return getattr(section, self.name)[new_key]


In [7]:
price_indices.head()
col_names = ['TIME'] + list(unit_costs_reformat.columns)
final_costs = pd.DataFrame(columns = col_names)

In [8]:
row_index= 0
for time in total_costs['TIME'].unique():
    for country in total_costs['GEO'].unique():
        values = [time,country]
        if country in unit_costs_reformat['GEO'].unique():
            for typeof in col_names[2:len(col_names)]:
                if (unit_costs_reformat.loc[unit_costs_reformat['GEO']==country,typeof].iloc[0] != ':'):
                    values.append(unit_costs_reformat.loc[unit_costs_reformat['GEO']==country,typeof].iloc[0])
                else:
                    available_values = unit_costs_reformat[['GEO',typeof]]
                    available_values = available_values.drop(available_values.loc[available_values[typeof]==":"].index).reset_index()
                    country_available = available_values.loc[0,'GEO']
                    price = available_values.loc[0,typeof]
                    if (time != 2019):
                        price_indices_year = price_indices[price_indices['TIME']== time]
                        price_indice_available_country = price_indices_year.loc[price_indices['GEO']==country_available,'Value'].iloc[0]
                        price_indice_required_country = price_indices_year.loc[price_indices['GEO']==country,'Value'].iloc[0]
                        if (price_indice_required_country == ":"):
                            price_indice_required_country = 100
                        price_required_country = float(price)*float(price_indice_required_country)/float(price_indice_available_country)
                        values.append(price_required_country)
                    else:
                        price_indices_year = price_indices[price_indices['TIME']== (time-1)]
                        price_indice_available_country = price_indices_year.loc[price_indices['GEO']==country_available,'Value'].iloc[0]
                        price_indice_required_country = price_indices_year.loc[price_indices['GEO']==country,'Value'].iloc[0]
                        if (price_indice_required_country == ":"):
                            price_indice_required_country = 100
                        price_required_country = float(price)*float(price_indice_required_country)/float(price_indice_available_country)
                        values.append(price_required_country)
            final_costs.loc[row_index] = values
            row_index = row_index + 1

        else:
            for typeof in col_names[2:len(col_names)]:
                available_values = unit_costs_reformat[['GEO',typeof]]
                available_values = available_values.drop(available_values.loc[available_values[typeof]==":"].index).reset_index()
                country_available = available_values.loc[0,'GEO']
                price = available_values.loc[0,typeof]
                if (time != 2019):
                    price_indices_year = price_indices[price_indices['TIME']== time]
                    price_indice_available_country = price_indices_year.loc[price_indices['GEO']==country_available,'Value'].iloc[0]
                    price_indice_required_country = price_indices_year.loc[price_indices['GEO']==country,'Value'].iloc[0]
                    if (price_indice_required_country == ":"):
                        price_indice_required_country = 100
                    price_required_country = float(price)*float(price_indice_required_country)/float(price_indice_available_country)
                    values.append(price_required_country)
                else:
                    price_indices_year = price_indices[price_indices['TIME']== (time-1)]
                    price_indice_available_country = price_indices_year.loc[price_indices['GEO']==country_available,'Value'].iloc[0]
                    price_indice_required_country = price_indices_year.loc[price_indices['GEO']==country,'Value'].iloc[0]
                    if (price_indice_required_country == ":"):
                        price_indice_required_country = 100
                    price_required_country = float(price)*float(price_indice_required_country)/float(price_indice_available_country)
                    values.append(price_required_country)
            
            final_costs.loc[row_index] = values
            row_index = row_index + 1
final_costs.to_csv('FinalUnitcosts.csv',index=False)